# CAEM-GBDT

CAEM-GBDT: a cancer subtype identifying method using Multi-omics data and convolutional autoencoder network

# import packages

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Reshape
from tensorflow.keras import Input
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv1D,Conv1DTranspose,BatchNormalization,Activation,multiply,MaxPool1D,Dropout,GRU,Flatten,Dense,UpSampling1D
from tensorflow.keras import Model,datasets
import os
import tensorflow.keras as keras
from keras import losses
from sklearn.metrics import f1_score, precision_score, recall_score
import os
import tensorflow.keras.backend as kb
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
import warnings
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.svm as svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# loading data and data preprocessing

In [ ]:
data = np.loadtxt('E:/多组学数据/datasets/txt/Breast/Breast_gene.txt',dtype=np.str)
data = np.array(data).astype(np.float32)
data.mean(axis=1)
data = data - data.mean(axis=0,keepdims=True)
data = data / np.sqrt(data.var(axis=0,keepdims = True))
data.var(axis=0)


data1 = np.loadtxt('E:/多组学数据/datasets/txt/Breast/Breast_Methy.txt',dtype=np.str)
data1 = np.array(data1).astype(np.float32)
data1.mean(axis=1)
data1 = data1 - data1.mean(axis=0,keepdims=True)
data1 = data1 / np.sqrt(data1.var(axis=0,keepdims = True))
data1.var(axis=0)


data2 = np.loadtxt('E:/多组学数据/datasets/txt/Breast/Breast_Mirna.txt',dtype=np.str)
data2 = np.array(data2).astype(np.float32)
data2.mean(axis=1)
data2 = data2 - data2.mean(axis=0,keepdims=True)
data2 = data2 / np.sqrt(data2.var(axis=0,keepdims = True))
data2.var(axis=0)


Data = tf.concat([data,data1,data2],axis=1)
Data = np.array(Data)
Data.mean(axis=1)
Data = Data - Data.mean(axis=0,keepdims=True)
Data = Data / np.sqrt(Data.var(axis=0,keepdims = True))
Data.var(axis=0)
Data.shape

(104,41262)

In [ ]:
tag = np.loadtxt('E:/多组学数据/datasets/txt/Breast/Breast_Label.txt',dtype=np.str)
tag = np.array(tag).astype(np.float32)
tag.shape

(104,)

# Gelu activation function

In [ ]:
def gelu(x):
            return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

# convolutional block attention module

In [ ]:
def cbam_block(cbam_feature, ratio=8,kernel_size = 3):

    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature,kernel_size)
    return cbam_feature

def channel_attention(input_feature, ratio=8):

    channel = input_feature.shape[-1]
    filters = max(1, int(channel//ratio))
    shared_layer_one = tf.keras.layers.Dense(filters,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = tf.keras.layers.Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(input_feature)    
    avg_pool = tf.keras.layers.Reshape((1,1,channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = tf.keras.layers.GlobalMaxPooling1D()(input_feature)
    max_pool = tf.keras.layers.Reshape((1,1,channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)
   

    cbam_feature = tf.keras.layers.Add()([avg_pool,max_pool])
    cbam_feature = tf.keras.layers.Activation('sigmoid')(cbam_feature)


    return multiply([input_feature, cbam_feature])
def spatial_attention(input_feature,kernel_siz):
    kernel_size = kernel_siz

    channel = input_feature.shape[-1]
    cbam_feature = input_feature

    avg_pool = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    max_pool = tf.keras.layers.Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    concat = tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool])
    cbam_feature = tf.keras.layers.Conv1D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)	

    return multiply([input_feature, cbam_feature])


# Convolutional Autoencoder

In [ ]:
class MyModel(tf.keras.Model):
    

    def __init__(self):
        super (MyModel,self).__init__()
        #expression
        self.f1 = tf.keras.layers.Dense(512,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.f2 = tf.keras.layers.Conv1D(filters = 16,kernel_size = 3,padding='same',strides = 2,activation=gelu)
        self.f3 = tf.keras.layers.MaxPool1D(pool_size = 2)
        self.f4 = tf.keras.layers.Conv1D(filters = 4,kernel_size = 3,padding='same',strides = 2,activation=gelu)
        self.f5 = tf.keras.layers.MaxPool1D(pool_size = 4)
        self.f6 = tf.keras.layers.Conv1DTranspose(filters = 4,kernel_size = 3,padding='same',strides =2,activation=gelu)
        self.f7 = tf.keras.layers.UpSampling1D(size = 4)
        self.f8 = tf.keras.layers.Conv1DTranspose(filters = 16,kernel_size = 3,padding='same',strides =2,activation=gelu)
        self.f9 = tf.keras.layers.UpSampling1D(size = 2)
        self.flatten = tf.keras.layers.Flatten()
        self.f10 = tf.keras.layers.Dense(512,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.f11 = tf.keras.layers.Dense(17814,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        
        #methy
        self.f12 = tf.keras.layers.Dense(512,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.f13 = tf.keras.layers.Conv1D(filters = 16,kernel_size = 3,padding='same',strides = 2,activation=gelu)
        self.f14 = tf.keras.layers.MaxPool1D(pool_size = 2)
        self.f15 = tf.keras.layers.Conv1D(filters = 4,kernel_size = 3,padding='same',strides = 2,activation=gelu)
        self.f16 = tf.keras.layers.MaxPool1D(pool_size = 4)
        self.f17 = tf.keras.layers.Conv1DTranspose(filters = 4,kernel_size = 3,padding='same',strides =2,activation=gelu)
        self.f18 = tf.keras.layers.UpSampling1D(size = 4)
        self.f19 = tf.keras.layers.Conv1DTranspose(filters = 16,kernel_size = 3,padding='same',strides =2,activation=gelu)
        self.f20 = tf.keras.layers.UpSampling1D(size = 2)
        self.flatten = tf.keras.layers.Flatten()
        self.f21 = tf.keras.layers.Dense(512,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.f22 = tf.keras.layers.Dense(23094,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        
        #mRNA
        self.f23 = tf.keras.layers.Dense(128,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.f24 = tf.keras.layers.Conv1D(filters = 4,kernel_size = 3,padding='same',strides = 2,activation=gelu)
        self.f25 = tf.keras.layers.MaxPool1D(pool_size = 4)
        self.f26 = tf.keras.layers.Conv1DTranspose(filters = 4,kernel_size = 3,padding='same',strides =2,activation=gelu)
        self.f27 = tf.keras.layers.UpSampling1D(size = 4)
        self.flatten = tf.keras.layers.Flatten()
        self.f28 = tf.keras.layers.Dense(128,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
        self.f29 = tf.keras.layers.Dense(354,activation = gelu,kernel_regularizer=tf.keras.regularizers.l2())
    def call(self,x):
        input1 = x[:,0:17814]
        input2 = x[:,17814:40908]
        input3 = x[:,40908:]
        #expression
        x = self.f1(input1)
        x = Reshape((512,1))(x)
        x = self.f2(x)
        x = self.f3(x)
        x = self.f4(x)
        encode1 = self.f5(x)
        x = self.f6(encode1)
        x = self.f7(x)
        x = self.f8(x)
        x = self.f9(x)
        x = self.flatten(x)
        x = self.f10(x)
        decode1 = self.f11(x)
        
        #methy
        x = self.f12(input2)
        x = Reshape((512,1))(x)
        x = self.f13(x)
        x = self.f14(x)
        x = self.f15(x)
        encode2 = self.f16(x)
        x = self.f17(encode2)
        x = self.f18(x)
        x = self.f19(x)
        x = self.f20(x)
        x = self.flatten(x)
        x = self.f21(x)
        decode2 = self.f22(x)
        
        #mRNA
        x = self.f23(input3)
        x = Reshape((128,1))(x)
        x = self.f24(x)
        encode3 = self.f25(x)
        x = self.f26(encode3)
        x = self.f27(x)
        x = self.flatten(x)
        x = self.f28(x)
        decode3 = self.f29(x)
        z = tf.concat([encode1,encode2,encode3],axis=1)
        c = cbam_block(z)
        d = self.flatten(c)
        return d,decode1,decode2,decode3
    
    
model = MyModel()
model.build(input_shape=(104,41262))
model.call(Input(shape=(41262,)))

model.summary()

# Training and testing

In [ ]:
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits,gene,methy,mrna = model(x, training=True)
        recon_loss1 = losses.mean_squared_error(x[:,0:17814],gene)
        recon_loss2 = losses.mean_squared_error(x[:,17814:40908],methy)
        recon_loss3 = losses.mean_squared_error(x[:,40908:],mrna)
        loss_value = recon_loss1 + recon_loss2 + recon_loss3
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    return loss_value,logits


def val_step(x, y):
    val_logits,gene,methy,mrna = model(x, training=False)

In [ ]:
kf = KFold(n_splits=4)
epos = 100
for epo in range(epos):
    print("\nStart of epoch %d" % (epo,))
    loss,log=train_step(data[0:78,:], tag[0:78])
    log = np.array(log)
    
    X_train,X_test,y_train,y_test = train_test_split(log,tag[0:78],test_size=0.25,random_state=20)
    
    #modelknn=KNeighborsClassifier(n_neighbors=10)
    #modelknn.fit(X_train,y_train)
    #y_pred = modelknn.predict(X_test)
    
    #modeldeep = CascadeForestClassifier(random_state=0)
    #modeldeep.fit(X_train,y_train)
    #y_pred = modeldeep.predict(X_test)
    
    #modelrf = RandomForestClassifier(random_state=0)
    #modelrf.fit(X_train,y_train)
    #y_pred = modelrf.predict(X_test)
    
    #modelsvm = svm.SVC(kernel='linear',probability=True,random_state=20)
    #modelsvm.fit(X_train,y_train)
    #y_pred = modelsvm.predict(X_test)
    
    gbm= GradientBoostingClassifier(learning_rate=0.1,n_estimators=300,max_depth=3,min_samples_leaf =5, min_samples_split =5, max_features='sqrt',subsample=0.8,random_state=10)
    gbm.fit(X_train,y_train)
    y_pred = gbm.predict(X_test)

 
    accuracy = accuracy_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred, average='macro' )
    p = precision_score(y_test,y_pred, average='macro')
    r = recall_score(y_test,y_pred, average='macro')
    
    print("accuracy: %.3f" % (float(accuracy),))
    print("Precision：%.3f",p)
    print("Recall：%.3f",r)
    print("F1：%.3f",f1)
 


    val_step(data[78:,:], tag[78:])


Start of epoch 21
accuracy: 0.950
Precision： 0.976
Recall： 0.889
F1： 0.930